 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 5: Optimisation avec contraintes et calcul variationnel
Tangi Migot et Paul Raynaud

In [46]:
using Pkg
Pkg.activate(".") #Accède au fichier Project.toml
Pkg.add("NLPModels")
Pkg.add("NLPModels")
Pkg.add("NLPModelsJuMP")
Pkg.add("NLPModelsIpopt")
Pkg.instantiate()
Pkg.instantiate()
Pkg.status()

  Activating project at `c:\Users\jgpal\OneDrive\Documents\GitHub\MTH8408-Hiv24\lab5_JP`


   Resolving package versions...


  No Changes to `C:\Users\jgpal\OneDrive\Documents\GitHub\MTH8408-Hiv24\lab5_JP\Project.toml`
  No Changes to `C:\Users\jgpal\OneDrive\Documents\GitHub\MTH8408-Hiv24\lab5_JP\Manifest.toml`

Precompiling project...


  ? PDENLPModels
   Resolving package versions...


...

  ? PDENLPModels


Precompiling project...


  ? PDENLPModels


Status `C:\Users\jgpal\OneDrive\Documents\GitHub\MTH8408-Hiv24\lab5_JP\Project.toml`


  [54578032] ADNLPModels v0.7.0
⌅ [56d4f2e9] Gridap v0.15.5
  [7073ff75] IJulia v1.24.2
  [ba0b0d4f] Krylov v0.9.5
  [a4795742] NLPModels v0.20.0
  [f4238b75] NLPModelsIpopt v0.10.1
  [792afdf1] NLPModelsJuMP v0.12.5
  [80da258d] PDENLPModels v0.3.4
  [ff4d7338] SolverCore v0.3.7
  [37e2e46d] LinearAlgebra
  [56ddb016] Logging
  [de0858da] Printf
  [8dfed614] Test
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1



In [47]:
using Krylov, LinearAlgebra, Logging, NLPModels, NLPModelsIpopt, Printf, SolverCore, Test

In [48]:
using PDENLPModels, Gridap

## Quelques commentaires en Julia

### Les kwargs: choix optionnels

Dans le projet du dernier labo, une des questions demandait d'ajouter une option pour utiliser la fonction `lsmr` ou `lsqr`. C'est le cas typique d'arguments optionnels:
- On veut proposer un choix par défaut à l'utilisateur, par exemple `lsqr`;
- On veut laisser la possibilité à l'utilisateur de changer;
- On voudrait aussi pouvoir ajouter d'autres par la suite (sans avoir à tout modifier).

In [49]:
function dsol(A, b, ϵ; solver :: Function = lsqr)
    (d, stats) = solver(A, b, atol = ϵ)
    return d
end

dsol (generic function with 1 method)

A noter que l'on donne des valeurs par défaut aux arguments qui apparaissent après le `;`.

## Exercice 1: Pénalité quadratique pour les ADNLPModels

Dans cet exercice, on va étudier une version simple d'une méthode de pénalité quadratique pour les problèmes d'optimisation avec contraintes d'égalité.
```math
min f(x) s.à c(x) = 0.
```
Dans les labos précédents, on a déjà utilisé un NLPModel particulier, le ADNLPModel:

In [50]:
using ADNLPModels, LinearAlgebra, Test
fH(x) = (x[2]+x[1].^2-11)^2 + (x[1]+x[2].^2-7)^2
x0H = [10., 20.]
cH(x) = [x[1]-1]
himmelblau = ADNLPModel(fH, x0H, cH, [0.], [0.])

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  ForwardDiffADJprod,
  ForwardDiffADJtprod,
  ForwardDiffADJacobian,
  ForwardDiffADHessian,
  ForwardDiffADGHjvprod,
}
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ████████████████████ 1     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ████████████████████ 1     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
     

Attention: dans toute la suite de l'exercice on suppose que les bornes sur les contraintes `nlp.meta.lcon` et `nlp.meta.ucon` sont 0 pour simplifier.

### Question 1: Transformer un ADNLPModel en un problème pénalisé
Coder la fonction `quad_penalty_adnlp` qui prend en entrée un ADNLPModel, et un paramètre ρ et qui retourne un nouveau ADNLPModel qui correspond au problème sans contrainte:
$$
\min_x f(x) + \frac{\rho}{2}\|c(x)\|^2.
$$
Remarque: on peut accèder aux fonctions f et c par `NLPModels.obj()` et `NLPModels.cons()`.

In [111]:
using ADNLPModels #Pkg.add("ADNLPModels")
using NLPModels, NLPModelsIpopt



function quad_penalty_adnlp(nlp :: ADNLPModel, ρ :: Real)
    # TODO
    f = x -> obj(nlp,x) + ρ/2 .* norm(cons(nlp,x))^2
    nlp_quad = ADNLPModel(f, nlp.meta.x0)
   return nlp_quad
end

function quad_penalty_adnlp(nlp :: MathOptNLPModel, ρ :: Real)
    # TODO
    f = x -> obj(nlp,x) + ρ/2 .* norm(cons(nlp,x))^2
    nlp_quad = ADNLPModel(f, nlp.meta.x0)
   return nlp_quad
end

quad_penalty_adnlp (generic function with 2 methods)

In [94]:
#Faire des tests pour vérifier que ça fonctionne.

himmelblau_quad = quad_penalty_adnlp(himmelblau, 1)
@test himmelblau_quad.meta.ncon == 0
@test obj(himmelblau_quad, zeros(2)) == 170.5

stats = ipopt(himmelblau_quad)



This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.7433050e+05 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

"Execution stats: first-order stationary"

In [95]:
stats.solution

2-element Vector{Float64}:
 2.9678888605539164
 2.0186523682501782

In [114]:
#Ajouter au moins un autre test similaire avec des contraintes.
using JuMP 
using NLPModels, LinearAlgebra, NLPModelsJuMP, NLPModelsIpopt

m = Model(Ipopt.Optimizer)
@variable(m, x[1:2])
x0 = [10. ,20.]
set_start_value.(x, x0) 
@NLobjective(m, Min, sum((x[i] for i in 1:2)))#sum((x[i]^2) for i in 1:2))
@constraint(m, x[1]==2)

nlp = MathOptNLPModel(m)



new_quad = quad_penalty_adnlp(nlp, 1)
@test new_quad.meta.ncon == 0

stats = ipopt(new_quad)


MethodError: MethodError: no method matching Float64(::ForwardDiff.Dual{ForwardDiff.Tag{var"#122#123"{MathOptNLPModel, Int64}, Float64}, Float64, 2})

Closest candidates are:
  (::Type{T})(::Real, !Matched::RoundingMode) where T<:AbstractFloat
   @ Base rounding.jl:207
  (::Type{T})(::T) where T<:Number
   @ Core boot.jl:792
  Float64(!Matched::IrrationalConstants.Log2π)
   @ IrrationalConstants C:\Users\jgpal\.julia\packages\IrrationalConstants\vp5v4\src\macro.jl:112
  ...


In [ ]:
# Ajouter un test au cas ou `nlp.meta.lcon` ou `nlp.meta.ucon` ont des composantes differentes de 0.

### Question 2: KKT
Coder une fonction `KKT_eq_constraint(nlp :: AbstractNLPModel, x, λ)` qui vérifie si le point `x` avec multiplicateur de Lagrange `λ` satisfait les conditions KKT d'un problème avec contraintes d'égalités.

In [82]:
function KKT_eq_constraint(nlp :: AbstractNLPModel, x, λ)
   # TODO
   constr = cons(nlp,x)
   ∇c= grad(constr,x)
   ∇f = grad(nlp,x)
   if (∇f == dot(λ,∇c)) & (constr == zeros(nlp.nvar))
      kkt_bool= true
   else
      kkt_bool= false
   end
   return kkt_bool
end

KKT_eq_constraint (generic function with 1 method)

In [ ]:
#test


### Question 3: méthode de pénalité quadratique

In [81]:
using NLPModelsIpopt

In [ ]:
function quad_penalty(nlp      :: AbstractNLPModel,
                      x        :: AbstractVector; 
                      ϵ        :: AbstractFloat = 1e-3,
                      η        :: AbstractFloat = 1e6, 
                      σ        :: AbstractFloat = 2.0,
                      max_eval :: Int = 1_000, 
                      max_time :: AbstractFloat = 60.,
                      max_iter :: Int = typemax(Int64)
                      )
    ##### Initialiser cx et gx au point x;
    cx = cons(nlp,x) # Initialiser la violation des contraintes
    gx = grad(nlp, x) # Initialiser le gradient
    ######################################################
    normcx = normcx_old = norm(cx)

    ρ = 1.

    iter = 0    

    el_time = 0.0
    tired   = neval_cons(nlp) > max_eval || el_time > max_time
    status  = :unknown

    start_time = time()
    too_small  = false
    normdual   = norm(gx) #exceptionnellement on ne va pas vérifier toute l'optimalité au début.
    optimal    = max(normcx, normdual) ≤ ϵ
    
    nlp_quad   = quad_penalty_adnlp(nlp, ρ)

    @info log_header([:iter, :nf, :primal, :status, :nd, :Δ],
    [Int, Int, Float64, String, Float64, Float64],
    hdr_override=Dict(:nf => "#F", :primal => "‖F(x)‖", :nd => "‖d‖"))

    while !(optimal || tired || too_small)

        #Appeler Ipopt pour résoudre le problème pénalisé en partant du point x0 = x.
        #utiliser l'option print_level = 0 pour enlever les affichages d'ipopt.
        stats = ipopt(nlp_quad) #...
        ################################################
      
        if stats.status == :first_order
            ###### Mettre à jour cx avec la solution renvoyé par Ipopt 
            """
            https://jso.dev/SolverCore.jl/dev/reference/#SolverCore.GenericExecutionStats
            """
            x = stats.solution #TODO
            cx = cons(nlp,x) #TODO
            ##########################################################
            normcx_old = normcx
            normcx = norm(cx)
        end
        
        if normcx_old > 0.95 * normcx
            ρ *= σ
        end

        @info log_row(Any[iter, neval_cons(nlp), normcx, stats.status])
        
        nlp_quad   = quad_penalty_adnlp(nlp, ρ)

        el_time      = time() - start_time
        iter   += 1
        many_evals   = neval_cons(nlp) > max_eval
        iter_limit   = iter > max_iter
        tired        = many_evals || el_time > max_time || iter_limit || ρ ≥ η
        ##### Utiliser la réalisabilité dual renvoyé par Ipopt pour `normdual`
        normdual     = stats.dual_feasibility # TODO
        ###################################################################
        optimal      = max(normcx, normdual) ≤ ϵ
    end

    status = if optimal 
        :first_order
    elseif tired
        if neval_cons(nlp) > max_eval
            :max_eval
        elseif el_time > max_time
            :max_time
        elseif iter > max_iter
            :max_iter
        else
            :unknown_tired
        end
    elseif too_small
        :stalled
    else
        :unknown
    end

    return GenericExecutionStats(nlp, status = status, solution = x,
                                 objective = obj(nlp, x),
                                 primal_feas = normcx,
                                 dual_feas = normdual,
                                 iter = iter, 
                                 elapsed_time = el_time,
                                 solver_specific = Dict(:penalty => ρ))
end

In [ ]:
#Faire des tests pour vérifier que ça fonctionne.
stats = quad_penalty(himmelblau, x0H)
@test stats.status == :first_order
@test stats.solution ≈ [1.0008083416169895, 2.709969135758311] atol=1e-2
@test norm(cons(himmelblau, stats.solution)) ≈ 0. atol=1e-3

Vérifier que la solution rendue vérifie les conditions KKT avec la fonction de la question précédente.

In [ ]:
# TODO
KKT_eq_constraint(nlp = himmelblau, stats.solution, λ)

In [ ]:
#Fichier de tests à demander.


## Exercice 2: Calcul Variationnel

Dans cet exercice, on considère le problème de calcul variationnel suivant:
$$
\min \int_0^1 (\dot{x}(t)^2+2x(t)^2)e^t dt, \quad x(0)=0, x(1)=e - e^{-2}
$$

modélisé avec `PDENLPModels`.

In [ ]:
function cv_model(n :: Int)

  domain = (0,1) # set the domain
  partition = n
  model = CartesianDiscreteModel(domain,partition) # set discretization
    
  labels = get_face_labeling(model)
  add_tag_from_tags!(labels,"diri1",[2])
  add_tag_from_tags!(labels,"diri0",[1]) # boundary conditions

  order=1
  valuetype=Float64
  reffe = ReferenceFE(lagrangian, valuetype, order)
  V0 = TestFESpace(model, reffe; conformity=:H1, dirichlet_tags=["diri0","diri1"])
  U = TrialFESpace(V0,[0., exp(1)-exp(-2)])

  trian = Triangulation(model)
  degree = 2
  dΩ = Measure(trian,degree) # integration machinery

  # Our objective function
  w(x) = exp(x[1])
  function f(y)
    ∫((∇(y)⊙∇(y) + 2 * y * y) * w) * dΩ
  end

  xin = zeros(Gridap.FESpaces.num_free_dofs(U))
  nlp = GridapPDENLPModel(xin, f, trian, U, V0)
  return nlp
end

### Question 1: Résoudre
Résoudre le NLPModel généré par la fonction `cv_model` pour `n = 16` avec `ipopt` et afficher la solution (attention la solution rendue ne contient pas les valeurs aux bords qu'il faut rajouter).

In [ ]:
# TODO

### Question 2: Convergence en `n`
Afficher sur un même graphique la solution obtenue par `ipopt` pour plusieurs valeurs de `n`.

### Question 3: Comparer à la solution exacte

La solution exacte est $x(t)=e^t - e^{-2t}$ et la valeur optimale est $e^3 - 2e^{-3}+1$.